In [1]:
import warnings
warnings.filterwarnings("ignore")

# 교차 검증과 그리드 서치
- 머신러닝을 사용할 떄 모델의 정확도를 측정하기 위해 반드시 사용해야 하는 방법
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다.

In [2]:
import pandas as pd
wine = pd.read_csv('../Data/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
# feature, target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

### 검증세트 추가

In [4]:
# 전체 세트 중 훈련세트와 테스트 세트를 8:2로 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data,target,test_size=0.2, random_state=42
)

In [5]:
# 훈련세트 중 훈련세트와 검증 세트를 8:2 기준으로 분리한다.
sub_input, val_input, sub_target,val_target = train_test_split(
    train_input,train_target,test_size=0.2, random_state=42
)

In [6]:
# 훈련세트와 검증세트, 테스트 세트 구학
print("훈련세트 : ",sub_input.shape)
print("검증세트 : ",val_input.shape)
print("테스트세트 : ",test_input.shape)

훈련세트 :  (4157, 3)
검증세트 :  (1040, 3)
테스트세트 :  (1300, 3)


In [7]:
# 훈련세트와 검증세트를 결정트리로 모델 만들기
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target )
print("train score :", dt.score(sub_input, sub_target))
print("valid score :", dt.score(val_input, val_target))

train score : 0.9971133028626413
valid score : 0.864423076923077


---
### 교차 검증


In [8]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
scores

{'fit_time': array([0.00775313, 0.00697303, 0.00650692, 0.00703597, 0.00628304]),
 'score_time': array([0.00109172, 0.00088596, 0.00071716, 0.00084496, 0.00085306]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [9]:
import numpy as np
np.mean(scores['test_score'])

0.855300214703487

---
# k- fold : 분할기를 사용한 교차 검증

In [12]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold()        # 기본 n_splits => 5 
scores = cross_validate(dt, train_input, train_target,cv=splitter) 
# cv = cross validator
scores

{'fit_time': array([0.00692415, 0.00728798, 0.00722599, 0.00669217, 0.0068121 ]),
 'score_time': array([0.00064397, 0.00079012, 0.00083995, 0.00063086, 0.00087285]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [13]:
np.mean(scores['test_score'])

0.855300214703487

In [16]:
# kfold의 fold를 10개로 나누어서 교차검즈
splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
# shuffle 과대적합 방지 
scores = cross_validate(dt, train_input, train_target,cv=splitter) 
np.mean(scores['test_score'])

0.8574181117533719

In [15]:
splitter = StratifiedKFold(n_splits=10)
scores = cross_validate(dt, train_input, train_target,cv=splitter) 
np.mean(scores['test_score'])

0.8616407292129834